In [ ]:
import sys
%load_ext autoreload
%reload_ext autoreload
%autoreload 2  

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from main_func import main
from ETL import read_data_manual, ETL
from utils.wandb_utils import fetch_artifacts, read_tracker_objects, download_wandb_artifact


In [ ]:
import wandb
secret_value = 'Your Secret_API_Key'
wandb.login(key=secret_value) 
api = wandb.Api()

In [ ]:
model_type = 'XGB' #? RF , XGB, 'LGBM'
MANUAL_EXP = True

# 1. Single Experiment (Manual)

In [ ]:
print('MANUAL_EXP:',MANUAL_EXP)
man_params = {'RF':None, 'XGB':None}

In [ ]:
man_params['RF'] = {
    'model_name' : 'RF',
    
    'target_symbol' : 'BTCUSDT',
    'trade_mode': 'long' ,   #"long" , "short"
    'trg_look_ahead' : 576,
    'trg_take_profit' : 4,
    'trg_stop_loss' : 2,

    'strg_look_ahead' : 576,
    'strg_take_profit' : 4,
    'strg_stop_loss' : 2,

   
    'n_rand_features': None,
    'save_model_mode' : None, # None, 'sample_train_size', 'last_train_size', 'all_data',
    'n_splits' : 5,
    'max_train_size' : 1900*288,
    'test_size' : 90*288,
    'train_test_gap':00*288,

    'parameters':{
        'n_estimators' : 100,
        'max_depth': 5,
        'max_features' : 64,
        'max_samples' : 0.5,
        'class_weight': {0:1,1:1}, # 'balanced'
        'random_state': 42,
        'n_jobs': -1
        }
              }

In [ ]:
man_params['XGB'] = {
    'model_name' : 'XGB',
    
    'target_symbol' : 'AAVEUSDT',
    'trade_mode': 'long' ,   #"long" , "short"
    'look_ahead' : 39779,
    'take_profit' : 6.986981818507871,
    'stop_loss' : 2.6261834603021663,

    # 'strg_look_ahead' : 2188,
    # 'strg_take_profit' : 2.011355809600868,
    # 'strg_stop_loss' : 36.2687608369979,

    'n_rand_features': None,
    'save_model_mode' : None, 
    'n_splits' : 2,
    'max_train_size' : 900*288,
    'test_size' : 90*288,
    # 'train_test_gap': 00*288,
    'leverage': 1,

    "parameters": {
        "colsample_bylevel": 0.8191288841145314,
        "colsample_bynode": 0.42760210022007134,
        "colsample_bytree": 0.541373857169406,
        'device' : "cuda", # 'cuda', "cpu"
        "predictor": "gpu_predictor",
        "gamma": 1, 
        "learning_rate": 0.2315076680363571, 
        "max_bin": 512,           
        "max_delta_step": 15, 
        "max_depth": 3,
        "max_leaves": 6,
        "min_child_weight": 6,
        "n_estimators": 454,
        "n_jobs" : -1,
        "objective": "binary:logitraw",
        'random_state': 42,
        "reg_alpha": 16,
        "reg_lambda": 2,
        "scale_pos_weight" : 9.186491797405976,
        "subsample": 0.5448006560715095,
        'tree_method':'hist',
        },
}

In [ ]:
import os
if MANUAL_EXP:
    exp_obj, exp_metadata, artifact_name = main(
        manual = True,
        man_params = man_params[model_type],
        dataset_path = f"{os.path.dirname(os.getcwd())}/Framework data/dataset.parquet",
        C5M_data_path = f"{os.path.dirname(os.getcwd())}/Framework data",
    )

In [ ]:
imp_fe = list(exp_obj.feature_importance[exp_obj.feature_importance.cv<.5].feature_name)
exp_obj.feature_importance[exp_obj.feature_importance.cv<.5].feature_name

In [ ]:
def plot_feature_importances_scatter(importance_df, top_n=10, figsize=(8, 6)):
    # Calculate mean importance and sort features
    mean_importance = importance_df['mean_importance']
    sorted_idx = mean_importance.argsort()[::-1]
    top_features = importance_df.iloc[sorted_idx[:top_n]]
    imp_cols = [f for f in importance_df if 'importance_fold' in f]
    # Prepare data for scatter plot
    feature_names = top_features['feature_name']
    importance_values = top_features[imp_cols].values.T  # Transpose for plotting

    # Calculate median importance
    median_importance = top_features['median_importance']

    # Create scatter plot
    fig, ax = plt.subplots(figsize=figsize)
    
    for i, feature in enumerate(feature_names):
        ax.scatter([i] * importance_values.shape[0], importance_values[:, i], 
                   alpha=0.6, label=feature)

    # Plot median line
    ax.plot(range(top_n), median_importance, color='red', linestyle='--', 
            linewidth=2, label='Median')
    
    # Add value labels on the median line
    for i, v in enumerate(median_importance):
        ax.text(i, v, f'{v:.3f}', ha='center', va='bottom', fontweight='bold')

    ax.set_xlabel('Features', fontsize=12)
    ax.set_ylabel('Feature Importance', fontsize=12)
    ax.set_title(f'Top {top_n} Feature Importances Across Folds with Median', fontsize=14)
    ax.set_xticks(range(top_n))
    ax.set_xticklabels(feature_names, rotation=90, ha='right')
#     ax.legend(title='Features', bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plot_feature_importances_scatter(exp_obj.feature_importance)

### Save manual object on W&B as Artifact

In [ ]:
if MANUAL_EXP:
    # wandb.init(settings=wandb.Settings(init_timeout=5))
    wandb.init( project = 'ultimate_models')
    obj_artifact = wandb.Artifact(artifact_name, 'manual_experiment', metadata = exp_metadata)
    obj_artifact.add_file(local_path=exp_obj.store_name+'.zip', name=exp_obj.store_name+'.zip')
    wandb.log_artifact(obj_artifact)

In [ ]:
if MANUAL_EXP:
    wandb.finish()

## Save imp_features (when we have RANDOM features)

In [ ]:
if MANUAL_EXP:
    upload_imp_features = True
    symbol = man_params[model_type]['target_symbol']
    trade_mode = man_params[model_type]['trade_mode']
    look_ahead = man_params[model_type]['trg_look_ahead']
    take_profit = man_params[model_type]['trg_take_profit']
    stop_loss = man_params[model_type]['trg_stop_loss']

In [ ]:
if MANUAL_EXP and upload_imp_features :
    print('save_imp_features ')
    import pickle as pkl
    the_name = f"df_feature_imp_{trade_mode}_{symbol}_M{look_ahead}_TP{take_profit}_SL{stop_loss}.pkl"
    file = open(the_name, 'wb')
    pkl.dump(exp_obj.feature_importance,file)
    file.close()
    
    imp_artifact_name = artifact_name + '_imp_features'
    wandb.init( project = 'forex', job_type="feature_selection")
    imp_artifact = wandb.Artifact(imp_artifact_name, 'feature_selection', metadata = exp_metadata)
    imp_artifact.add_file(local_path = the_name,name = the_name)
    print(f"--> dataset size MB: {imp_artifact.size / 1000000}")
    wandb.log_artifact(imp_artifact)
    wandb.finish()

# 2. Sweep Experiment using W&B

In [ ]:
sweep_configs = {}

sweep_configs['XGB'] = {
    "method": "bayes",
    "metric": {
        "name": "profit_percent_mean_valid&test", 
        "goal": "maximize"   
    },
        "parameters": {
        'model_name' : {"values": ['XGB']},
        
        'target_symbol' : {"values": ['AAVEUSDT']}, 
        'trade_mode' : {"values": ['long']}, 
        "look_ahead": {'distribution': 'int_uniform', 'min': 5.0, 'max': 7200.0},
        "take_profit": {'distribution': 'uniform', 'min': 1, 'max': 40.0},
        "stop_loss": {'distribution': 'uniform', 'min': 1, 'max': 40.0},
        
        # "strg_look_ahead": {'distribution': 'int_uniform', 'min': 60.0, 'max': 43200.0},
        # "strg_take_profit": {'distribution': 'uniform', 'min': 0.5, 'max': 40.0},
        # "strg_stop_loss": {'distribution': 'uniform', 'min':0.5, 'max': 40.0},
                
        'n_rand_features': {"values": [None]}, 
        'save_model_mode' : {"values": [None]}, 

        "early_stopping_rounds" :{"values": [None]}, 
        'n_splits' : {"values":[23]},
        'max_train_size' : {"values":[180*288]},
        'test_size' : {"values":[25*288]},
        # 'train_test_gap' : {"values": [0]},
        # 'step_size': {"values": [900*288]},
        'leverage': {"values": [1]},
        



        "colsample_bynode": {'distribution': 'uniform', 'min': 0.4, 'max': 0.9},
        "colsample_bylevel": {'distribution': 'uniform', 'min': 0.4, 'max': 0.9},
        "colsample_bytree": {'distribution': 'uniform', 'min': 0.4, 'max': 0.9},
        'device' : {"values": ["cuda"]}, 
        "predictor": {"values": ["gpu_predictor"]},
        'eval_metric':{"values": ["aucpr"]},
        "gamma": {'distribution': 'int_uniform', 'min': 1.0, 'max': 20.0}, 
        "learning_rate": {'distribution': 'uniform', 'min': 0.01, 'max': 0.9},
        "max_bin": {"values": [256, 512]},            
        "max_delta_step": {'distribution': 'int_uniform', 'min': 1.0, 'max': 20.0}, 
        "max_depth": {'distribution': 'int_uniform', 'min': 1.0, 'max': 6.0},
        "max_leaves": {'distribution': 'int_uniform', 'min': 1.0, 'max': 20.0},
        "min_child_weight": {'distribution': 'int_uniform', 'min': 1.0, 'max': 20.0}, 
        "n_estimators": {'distribution': 'int_uniform', 'min': 100.0, 'max': 500.0},
        "n_jobs" : {"values":[-1]},
        "objective": {"values": ["binary:logitraw","binary:logistic"]},
        'random_state': {"values":[42]},
        "reg_lambda": {'distribution': 'int_uniform', 'min': 1.0, 'max': 20.0},
        "reg_alpha": {'distribution': 'int_uniform', 'min': 1.0, 'max': 20.0},
        "scale_pos_weight" : {'distribution': 'uniform', 'min': 0.001, 'max': 20.0},
        "subsample": {'distribution': 'uniform', 'min': 0.4, 'max': 0.9},
        },
}

In [ ]:
%%time

sweep_id = wandb.sweep(
    sweep_configs[model_type],

    project = 'backtestAAVEUSDT_Long')

#!!!!!!!!!!!! SWEEP ID !!!!!!!!!!!!!!!!!
wandb.agent(sweep_id = sweep_id, function = main, project = 'backtestAAVEUSDT_Long')
wandb.finish()

In [ ]:
#STOP SWEEP
if not MANUAL_EXP:
    print('Finishing The SWEEP ...')
    print(f'{1/0}')